In [1]:
import tensorflow as tf
import larq as lq
import numpy as np
import pandas as pd
import seaborn as sns
from binaryflow import quantizers
from binaryflow.layers import ABCNet,XnorNet
from binaryflow.block import BiRealNet
from binaryflow.layers.normalization import *

2022-08-04 17:27:39.059253: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-08-04 17:27:39.059286: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: fedora
2022-08-04 17:27:39.059292: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: fedora
2022-08-04 17:27:39.059381: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.57.0
2022-08-04 17:27:39.059397: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.57.0
2022-08-04 17:27:39.059403: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.57.0
2022-08-04 17:27:39.059650: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operat

In [2]:
  data_format="channels_last"
    
(X_train,y_train),(X_test,y_test)=tf.keras.datasets.mnist.load_data()
X_train=X_train.astype(dtype=np.float32)
y_train=tf.one_hot(y_train,10)

# All quantized layers except the first will use the same options

kwargs = dict(
              kernel_quantizers="ste_sign",
              input_quantizers="ste_sign",
              kernel_constraint="weight_clip",
              #kernel_params={"mu_initializer":tf.keras.initializers.RandomNormal(0,0.05)},
              use_bias=False
              )

Dense=ABCNet.ABCDense
Conv2D=ABCNet.ABCConv2D

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    #tf.keras.layers.GaussianNoise(stddev=4),
    ImageNormalizationLayer(),
    tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
    Dense(1024,activation="relu", **kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
    Dense(1024,activation="relu",**kwargs),
    tf.keras.layers.BatchNormalization(momentum=0.999,scale=False),
    Dense(10, **kwargs),
    tf.keras.layers.Activation("softmax")
])


model.compile(
    tf.keras.optimizers.Adam(lr=0.01, decay=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)


trained_model = model.fit(
    X_train, 
    y_train,
    batch_size=128, 
    epochs=30,
    validation_data=(X_test, tf.one_hot(y_test, 10)),
    shuffle=True
)

Epoch 1/30


/home/ramizouari/anaconda3/envs/tensorflow_2_9/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


 87/469 [====>.........................] - ETA: 14s - loss: 0.5890 - accuracy: 0.8333

KeyboardInterrupt: 

In [21]:
import json
json.dump(model.history.history,open("results/MNIST/ABCNet-Results-MNIST.","w"))

In [3]:
lq.models.summary(model)

+sequential stats----------------------------------------------------------------------------------------+
| Layer                      Input prec.     Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs  32-bit MACs |
|                                  (bit)                  x 1       x 1    (kB)                          |
+--------------------------------------------------------------------------------------------------------+
| flatten                              -   (-1, 784)        0         0       0           0            0 |
| image_normalization_layer            -   (-1, 784)        0         0       0           ?            ? |
| batch_normalization                  -   (-1, 784)        0      1568    6.12           0            0 |
| abc_dense                            1  (-1, 1024)  2408448      3072  306.00     2408448         3072 |
| batch_normalization_1                -    multiple        0      2048    8.00           ?            ? |
| abc_dense_1                        

In [5]:
model.layers[5].estimators[2].kernel

<QuantizedVariable 'abc_dense_1/kernel:0' shape=(1024, 1024) dtype=float32 quantizer=SteSign precision=1 numpy=
array([[ 0.08782385, -0.06336718, -0.0795977 , ..., -0.01897999,
         0.13732205,  0.0150758 ],
       [ 0.06247743, -0.00615585, -0.04191279, ..., -0.14396805,
        -0.07085136,  0.04433692],
       [ 0.01148298, -0.07169718, -0.09347291, ...,  0.01800714,
         0.0952584 , -0.03183747],
       ...,
       [ 0.06324833, -0.03321908,  0.03360074, ..., -0.03294454,
         0.0767939 ,  0.07613894],
       [ 0.00846735, -0.02115245, -0.04455905, ..., -0.00715694,
         0.1179852 , -0.12927361],
       [-0.04949906,  0.0377853 ,  0.05305236, ...,  0.04323835,
        -0.05108726,  0.03020819]], dtype=float32)>

In [8]:
print(.op_count("mac"))

None


In [33]:
lq.models.op_count_supported_layer_types=tuple(list(lq.models.op_count_supported_layer_types)+[ABCNet.ABCDense])

In [6]:
isinstance(model.layers[3].estimators[0], lq.models.op_count_supported_layer_types)

True

In [43]:
lq.models.mac_containing_layers

(larq.layers.QuantConv2D,
 larq.layers.QuantSeparableConv2D,
 larq.layers.QuantDepthwiseConv2D,
 larq.layers.QuantDense,
 keras.layers.convolutional.conv2d.Conv2D,
 keras.layers.convolutional.separable_conv2d.SeparableConv2D,
 keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D,
 keras.layers.core.dense.Dense,
 larq.layers.QuantConv1D,
 larq.layers.QuantSeparableConv1D,
 keras.layers.convolutional.conv1d.Conv1D,
 keras.layers.convolutional.separable_conv1d.SeparableConv1D)

In [45]:
lq.models.op_count_supported_layer_types

(keras.layers.reshaping.flatten.Flatten,
 keras.layers.normalization.batch_normalization.BatchNormalization,
 keras.layers.pooling.max_pooling2d.MaxPooling2D,
 keras.layers.pooling.average_pooling2d.AveragePooling2D,
 keras.layers.pooling.max_pooling1d.MaxPooling1D,
 keras.layers.pooling.average_pooling1d.AveragePooling1D,
 larq.layers.QuantConv2D,
 larq.layers.QuantSeparableConv2D,
 larq.layers.QuantDepthwiseConv2D,
 larq.layers.QuantDense,
 keras.layers.convolutional.conv2d.Conv2D,
 keras.layers.convolutional.separable_conv2d.SeparableConv2D,
 keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D,
 keras.layers.core.dense.Dense,
 larq.layers.QuantConv1D,
 larq.layers.QuantSeparableConv1D,
 keras.layers.convolutional.conv1d.Conv1D,
 keras.layers.convolutional.separable_conv1d.SeparableConv1D,
 binaryflow.layers.ABCNet.ABCDense)

In [9]:
profiler=lq.models.LayerProfile(model.layers[3].estimators[0])

In [12]:
profiler.output_shape

In [13]:
model.layers[3].estimators[0].output_shape

AttributeError: The layer "scaled_quant_dense" has never been called and thus has no defined output shape.

In [5]:
isinstance(,tf.keras.Model)

True

In [6]:
s=tf.keras.Sequential([])

In [7]:
s

In [8]:
type(s)

keras.engine.sequential.Sequential